## L1 and L2 Regularizations

In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv("C:\\Users\\user\\Downloads\\Amazon Sale Report.csv\\Amazon Sale Report.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_13800\369179484.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("C:\\Users\\user\\Downloads\\Amazon Sale Report.csv\\Amazon Sale Report.csv")


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

## Data Transformation

In [4]:
# first drop the most unwanted columns
DROP_COLS = [
    "index",
    "Order ID",           # pure identifier
    "SKU",                # high cardinality ID
    "promotion-ids",      # sparse + leakage prone
    "Unnamed: 22",
    "ASIN",
    "fulfilled-by",
    "ship-postal-code",
    "currency",
    "Size"
]

data = data.drop(columns=[c for c in DROP_COLS if c in data.columns])


In [5]:
# rename some columns
data.rename(columns={'Sales Channel ':'Sales Channel'}, inplace=True)

In [6]:
data.columns

Index(['Date', 'Status', 'Fulfilment', 'Sales Channel', 'ship-service-level',
       'Style', 'Category', 'Courier Status', 'Qty', 'Amount', 'ship-city',
       'ship-state', 'ship-country', 'B2B'],
      dtype='object')

In [7]:
# extract some important columsn from existing columns
data["Date"] = pd.to_datetime(data["Date"], dayfirst=True, errors="coerce")

data["order_day"] = data["Date"].dt.day_name()
data["order_month"] = data["Date"].dt.month_name()
data["order_weekday"] = data["Date"].dt.weekday
data["is_weekend"] = data["order_weekday"].isin([5,6]).astype(int)


# again drop the irrelevant columns
data = data.drop(columns=["Date"])


C:\Users\user\AppData\Local\Temp\ipykernel_13800\2998258809.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Date"] = pd.to_datetime(data["Date"], dayfirst=True, errors="coerce")


## Feature Engineering

In [8]:
# seperating numerical and categorical columns
numerical_cols = data.select_dtypes(include="number").columns
print("numerical_cols:\n", numerical_cols)
print()

categorical_cols = data.select_dtypes(exclude="number").columns
print("catgeorical_cols:\n", categorical_cols)

numerical_cols:
 Index(['Qty', 'Amount', 'order_weekday', 'is_weekend'], dtype='object')

catgeorical_cols:
 Index(['Status', 'Fulfilment', 'Sales Channel', 'ship-service-level', 'Style',
       'Category', 'Courier Status', 'ship-city', 'ship-state', 'ship-country',
       'B2B', 'order_day', 'order_month'],
      dtype='object')


In [9]:
numerical_cols = numerical_cols.drop(['Amount'])
numerical_cols

Index(['Qty', 'order_weekday', 'is_weekend'], dtype='object')

In [10]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler


In [11]:

# build a function -> general purpose -> to handle ml preprocessings
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

def build_preprocessor(num_cols, cat_cols, scale_numeric=True):
    
    num_steps = [
        ("imputer", SimpleImputer(strategy="median"))
    ]
    
    if scale_numeric:
        num_steps.append(("scaler", StandardScaler()))
    
    num_pipeline = Pipeline(num_steps)

    cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ])

    return ColumnTransformer([
        ("num", num_pipeline, num_cols),
        ("cat", cat_pipeline, cat_cols)
    ])


In [12]:
# for linear models -> scaling is on!
preprocessor = build_preprocessor(
    num_cols=numerical_cols,
    cat_cols=categorical_cols,
    scale_numeric=True
)

In [13]:

TARGET = "Amount"   # regression target

X = data.drop(columns=[TARGET])
y = data[TARGET]


In [14]:
# fill target with median values
y.fillna(value=data['Amount'].median(), inplace=True)


In [15]:
X.shape

(128975, 16)

In [16]:
y.shape

(128975,)

**split the data into training and testing data**

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


In [18]:
X_train.shape

(103180, 16)

In [19]:
y_train.shape

(103180,)

**apply prerocessor pipeline on training and testing inputs**

In [20]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)


In [21]:
X_train_processed.shape

(103180, 9470)

as the original training data is too heavy, so for prototyping and learning purpose, we can take samples to get actual insightsusing short data .

so, sample data -> train model -> real insights

In [22]:
# create bins for stratification (quantiles work best)
y_bins = pd.qcut(y_train, q=10, duplicates="drop")


In [23]:
X_train_sampled, _, y_train_sampled, _ = train_test_split(
    X_train_processed,
    y_train,
    test_size=0.9,          # keep 10% for fast training
    stratify=y_bins,
    random_state=42
)


In [24]:
X_train_sampled.shape

(10318, 9470)

## Model Training

#### with sampled data

a) building the ***Lasso (L1) Regularization***

In [31]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error


In [28]:
lasso = Lasso(alpha=0.1, max_iter=3000)
lasso.fit(X_train_sampled, y_train_sampled)


Lasso(alpha=0.1, max_iter=3000)

In [29]:
y_pred = lasso.predict(X_test_processed)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)


RMSE: 149.73802443274474


In [32]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

mae = mean_squared_error(y_test, y_pred)
print("MAE: ", mae)

mse = mean_squared_error(y_test, y_pred)
print("MSE: ", mse)

r2_score = r2_score(y_test, y_pred)
print("R2 score: ", r2_score)

MAE:  22421.475961021264
MSE:  22421.475961021264
R2 score:  0.7038831974405753


b) building the ***Ridge (L2) Regularization***

In [25]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1.0)
ridge.fit(X_train_sampled, y_train_sampled)


Ridge()

In [27]:
y_pred = ridge.predict(X_test_processed)


In [28]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

mae = mean_squared_error(y_test, y_pred)
print("MAE: ", mae)

mse = mean_squared_error(y_test, y_pred)
print("MSE: ", mse)

r2_score = r2_score(y_test, y_pred)
print("R2 score: ", r2_score)

rmse = np.sqrt(mse)

print("RMSE:", rmse)


MAE:  17436.79826470205
MSE:  17436.79826470205
R2 score:  0.7697150286629875
RMSE: 132.0484693765969


#### with whole data

a) building the ***Ridge Regularization*** - with whole data

In [34]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


In [35]:
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_processed, y_train)


Ridge()

In [36]:
y_pred_ridge = ridge.predict(X_test_processed)


In [37]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

print("R2:", r2_score(y_test, y_pred_ridge))
print("MAE:", mean_absolute_error(y_test, y_pred_ridge))


R2: 0.7977083470135472
MAE: 59.98147034556125


In [39]:

mse = mean_squared_error(y_test, y_pred_ridge)
print("MSE: ", mse)

rsme = np.sqrt(mse)
print("RMSE: ", rmse)

MSE:  15317.190363220901
RMSE:  132.0484693765969


b) building the ***Lasso Regularization*** - with whole data

In [40]:
from sklearn.linear_model import Lasso


In [41]:
# alpha controls the L1 penalty strength
lasso = Lasso(alpha=0.1)  
lasso.fit(X_train_processed, y_train)


Lasso(alpha=0.1)

In [42]:
y_pred_lasso = lasso.predict(X_test_processed)


In [43]:
mse = mean_squared_error(y_test, y_pred_lasso)
rmse = np.sqrt(mse)

print("R2:", r2_score(y_test, y_pred_lasso))
print("MSE:", mse)
print("RMSE:", rmse)


R2: 0.7118211739759999
MSE: 21820.425468344627
RMSE: 147.71738377166253


In [45]:
import pandas as pd

# get feature names from preprocessor
feature_names = preprocessor.get_feature_names_out()

# create DataFrame of features and Lasso coefficients
coef_df = pd.DataFrame({
    "feature": feature_names,
    "coefficient": lasso.coef_
}).sort_values(by="coefficient", key=abs, ascending=False)

# show top 10 most important features
print(coef_df.head(10))


                                feature  coefficient
15               cat__Fulfilment_Amazon  -630.493398
136                    cat__Style_J0011   571.888965
1232                  cat__Style_SET278   569.433886
342                    cat__Style_J0280   568.038761
19    cat__ship-service-level_Expedited   549.158178
323                    cat__Style_J0238   536.334575
347                    cat__Style_J0285   530.110741
313                    cat__Style_J0228   480.696043
139                    cat__Style_J0014   466.688341
343                    cat__Style_J0281   450.851237
